In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import time 
from pandas import Series
%matplotlib inline

# 定义文件全局变量
path = "/Users/skyamz/Desktop/JDD_Loan_Forecasting_Qualification/initdata/"
generate_path = "/Users/skyamz/Desktop/JDD_Loan_Forecasting_Qualification/generate_data/"


# 加载文件
file_order = pd.read_csv(path + "t_order.csv")
file_click = pd.read_csv(path + "t_click.csv")
file_user = pd.read_csv(path + "t_user.csv")
file_loan = pd.read_csv(path + "t_loan.csv")
file_loan_sum = pd.read_csv(path + "t_loan_sum.csv")

# 格式化时间
file_order['buy_time_format'] = pd.to_datetime(file_order['buy_time'])
file_loan['loan_time_format'] = pd.to_datetime(file_loan['loan_time'])

# 创建索引
file_order = file_order.set_index('buy_time_format')
file_loan = file_loan.set_index('loan_time_format')

#聚合用户订单月份数据
def user_order_month_aggregate(month):
    # 取月份数据
    file_order_month = file_order[month]
    # 重置索引
    file_order_month = file_order_month.reset_index()
    # 统计用户订单某个类目的价格、数量、折扣
    file_order_month_sum = file_order_month.groupby(['uid','cate_id'])['price','qty','discount'].sum()
    # 重置索引
    file_order_month_sum = file_order_month_sum.reset_index()
    # 合并2列
    file_order_month_sum['cate_id_and_count'] = file_order_month_sum['cate_id'].astype('str') + ':' +  file_order_month_sum['qty'].astype('str')
    # 统计用户下单类目
    file_order_month_sum_unique_cate_list = file_order_month_sum.groupby('uid')['cate_id_and_count'].unique()
    # 重置索引
    file_order_month_sum_unique_cate_list = file_order_month_sum_unique_cate_list.reset_index()
    # 统计用户订单全部价格、数量、折扣
    file_order_month_total = file_order_month.groupby(['uid'])['price','qty','discount'].sum()
    # 重置索引
    file_order_month_total = file_order_month_total.reset_index()
    # 将数据进行聚合操作
    user_order_month_aggregate = pd.merge(file_order_month_sum_unique_cate_list,file_order_month_total,how='outer',on='uid')
    return user_order_month_aggregate

# 索引各个月份数据
file_order_month_aggregate_11 = user_order_month_aggregate('2016-11')
file_order_month_aggregate_10 = user_order_month_aggregate('2016-10')
file_order_month_aggregate_09 = user_order_month_aggregate('2016-09')
file_order_month_aggregate_08 = user_order_month_aggregate('2016-08')

# 合并
file_click['pid_param'] = file_click['pid'].astype('str') +'_' + file_click['param'].astype('str')

# 用户点击统计
user_click_count = pd.DataFrame(file_click.groupby(['uid','pid_param'])['uid'].count())

# 重命名统计列
user_click_count.columns = ['click_count']

# 查询某个用户的点击统计
def user_click_count_serch(uid):
   user_click_count_serch = user_click_count.xs(uid, level='uid')
   return user_click_count_serch

# 查询某个参数的用户点击
def pid_and_param_click_count_serch(pid_param):
   pid_and_param_click_count_serch = user_click_count.xs(pid_param, level='pid_param')
   return pid_and_param_click_count_serch

# 查询用户的点击统计
# user_click_count_serch(1)

# 查询某个参数的用户点击
# pid_and_param_click_count_serch('8_1')

# 重置索引,并设置索引为uid
user_click_count_detail = user_click_count.reset_index().set_index('uid')

# 统计用户的总点击次数
user_click_count_total = pd.DataFrame(user_click_count.groupby('uid')['click_count'].sum())

# 重命名列名称
user_click_count_total.columns = ['click_count_total']

# 重置索引
user_click_count_total = user_click_count_total.reset_index()

# 合并2列为字典
user_click_count_detail['param_and_click'] = user_click_count_detail['pid_param'].astype('str') + ':' + user_click_count_detail['click_count'].astype('str')

# 合并分组
user_click_count_detail_aggregate = pd.DataFrame(user_click_count_detail.groupby('uid')['param_and_click'].unique())

# 重置索引
user_click_count_detail_aggregate = user_click_count_detail_aggregate.reset_index()

# 用户点击聚合
user_click_count_aggregate = pd.merge(user_click_count_detail_aggregate,user_click_count_total,how='outer',on='uid')

# 各月用户点击和订单数据汇总
user_click_count_and_order_month_aggregate_08 = pd.merge(user_click_count_detail_aggregate,file_order_month_aggregate_08,how='outer',on='uid')
user_click_count_and_order_month_aggregate_09 = pd.merge(user_click_count_detail_aggregate,file_order_month_aggregate_09,how='outer',on='uid')
user_click_count_and_order_month_aggregate_10 = pd.merge(user_click_count_detail_aggregate,file_order_month_aggregate_10,how='outer',on='uid')
user_click_count_and_order_month_aggregate_11 = pd.merge(user_click_count_detail_aggregate,file_order_month_aggregate_11,how='outer',on='uid')

# 重置索引
user_click_count_detail = user_click_count_detail.reset_index()

# 统计用户最大点击
user_click_count_detail_max = user_click_count_detail.loc[user_click_count_detail.groupby(['uid'])['click_count'].idxmax()].sort_values(by='uid')

# 查询每月订单最大数量
def file_order_month_max(month):
    file_order_month_sum = file_order[month].reset_index().groupby(['uid','cate_id'])['price','qty','discount'].sum().reset_index()
    file_order_month_sum.loc[file_order_month_sum.groupby(['uid'])['qty'].idxmax()].sort_values(by='uid')
    file_order_month_sum_max = file_order_month_sum.loc[file_order_month_sum.groupby(['uid'])['qty'].idxmax()].sort_values(by='uid')
    file_order_month_sum_max.columns = ['uid','cate_id_by_max_qty','price_by_max_qty','qty_max','discount_by_max_qty']
    return file_order_month_sum_max

# 查询点击最多次数和购买数量最大
def click_count_max_and_order_qty_max(month):
    click_count_max_and_order_qty_max = pd.merge(user_click_count_detail_max,file_order_month_max(month),how='outer',on='uid')
    click_count_max_and_order_qty_max = pd.merge(click_count_max_and_order_qty_max,file_user,how='outer',on='uid')
    return click_count_max_and_order_qty_max

# 统计贷款笔数
def loan_count(month):
    file_loan_count = pd.DataFrame(file_loan[month].groupby('uid')['uid'].count())
    file_loan_count.columns = ['loan_count']
    file_loan_count = file_loan_count.reset_index()
    file_loan_sum = pd.DataFrame(file_loan[month].groupby(['uid'])['loan_amount'].sum())
    file_loan_sum.columns = ['loan_sum']
    file_loan_sum = file_loan_sum.reset_index()
    file_loan_count_and_sum = pd.merge(file_loan_count,file_loan_sum,how='outer',on='uid')
    file_loan_count_and_sum['loan_sum_mean'] = file_loan_count_and_sum['loan_sum']/file_loan_count_and_sum['loan_count']
    return file_loan_count_and_sum

#打标用户当月是否借贷
def is_month_loan_user(month):
    loan_month = file_loan[month]
    is_loan_user = loan_month['uid'].unique()
    is_loan_user = pd.DataFrame(Series(is_loan_user))
    is_loan_user.columns = ['uid']
    is_loan_user['is_loan'] = 1
    return is_loan_user

#构建每个月的用户特征
def feature_month_uid(month,prefix_name):
    click_count_max_and_order_qty_max_tmp = click_count_max_and_order_qty_max(month)
    loan_count_tmp = loan_count(month)
    feature_month_uid = pd.merge(click_count_max_and_order_qty_max_tmp,loan_count_tmp,how='outer',on='uid')
    is_month_loan_user_tmp = is_month_loan_user(month)
    feature_month_uid = pd.merge(feature_month_uid,is_month_loan_user_tmp,how='outer',on='uid')
    feature_month_uid['diff'] = feature_month_uid['limit'] - feature_month_uid['loan_sum_mean']
    #one_hot_pid_param = pd.get_dummies(feature_month_uid['pid_param'])
    #one_hot_cate_id_by_max_qty = pd.get_dummies(feature_month_uid['cate_id_by_max_qty'])
    #one_hot_pid_param['uid'] = feature_month_uid['uid']
    #one_hot_cate_id_by_max_qty['uid'] = feature_month_uid['uid']
    #one_hot_all = pd.merge(one_hot_pid_param,one_hot_cate_id_by_max_qty,how = 'outer',on='uid')
    #feature_month_uid = pd.merge(feature_month_uid,one_hot_all,how='outer',on='uid')
    del feature_month_uid['active_date']
    del feature_month_uid['param_and_click']
    del feature_month_uid['pid_param']
    del feature_month_uid['cate_id_by_max_qty']
    feature_month_uid.columns = [col+prefix_name for col in feature_month_uid.columns.values.astype('str')]
    feature_month_uid = feature_month_uid.rename(columns={"uid"+prefix_name:"uid"})
    feature_month_uid = feature_month_uid.where(feature_month_uid.notnull(), 0)
    return feature_month_uid

#只查询贷款贷款
def feature_month_uid_is_loan(month,prefix_name):
    feature_month_uid_is_loan = feature_month_uid(month,prefix_name)
    feature_month_uid_is_loan = feature_month_uid_is_loan[feature_month_uid_is_loan['is_loan'+prefix_name] == 1.0]
    feature_month_uid_is_loan['loan_sum'+prefix_name] = feature_month_uid_is_loan['loan_sum'+prefix_name]/10
    feature_month_uid_is_loan['limit'+prefix_name] = feature_month_uid_is_loan['limit'+prefix_name]/10
    feature_month_uid_is_loan['loan_sum_mean'+prefix_name] = feature_month_uid_is_loan['loan_sum_mean'+prefix_name]/10
    return feature_month_uid_is_loan

# 取各月数据
feature_month_uid_8 = feature_month_uid('2016-08','_8')
feature_month_uid_9 = feature_month_uid('2016-09','_9')
feature_month_uid_10 = feature_month_uid('2016-10','_10')
feature_month_uid_11 = feature_month_uid('2016-11','_11')

feature_month_uid_is_loan_8 = feature_month_uid_is_loan('2016-08','_8')
feature_month_uid_is_loan_9 = feature_month_uid_is_loan('2016-09','_9')
feature_month_uid_is_loan_10 = feature_month_uid_is_loan('2016-10','_10')
feature_month_uid_is_loan_11 = feature_month_uid_is_loan('2016-11','_11')

# 合并每个月数据
feature_month_uid_8_9 = pd.merge(feature_month_uid_8,feature_month_uid_9,how='outer',on='uid')
feature_month_uid_10_11 = pd.merge(feature_month_uid_10,feature_month_uid_11,how='outer',on='uid')
feature_month_uid_all = pd.merge(feature_month_uid_8_9,feature_month_uid_10_11,how='outer',on='uid')


feature_month_uid_is_loan_8_9 = pd.merge(feature_month_uid_is_loan_8,feature_month_uid_is_loan_9,how='outer',on='uid')
feature_month_uid_is_loan_10_11 = pd.merge(feature_month_uid_is_loan_10,feature_month_uid_is_loan_11,how='outer',on='uid')
feature_month_uid_is_loan_all = pd.merge(feature_month_uid_is_loan_8_9,feature_month_uid_is_loan_10_11,how='outer',on='uid')
        
    

# 替换所有nan为0
feature_month_uid_all_replace_none = feature_month_uid_is_loan_all.where(feature_month_uid_is_loan_all.notnull(), 0)

# 按uid从小到大排序
feature_month_uid_all_replace_none = feature_month_uid_all_replace_none.sort_values(by='uid')

def build_train_set():
    loan_and_user = pd.merge(file_loan_sum,file_user,how='outer',on='uid')
    loan_and_user = loan_and_user.sort_values(by='uid')
    loan_and_user['loan_sum'] = loan_and_user['loan_sum']/10
    del loan_and_user['age']
    del loan_and_user['sex']
    del loan_and_user['limit']
    del loan_and_user['active_date']
    # 取前80%作为训练集
    feature_month_is_loan_sum = pd.merge(feature_month_uid_all_replace_none , loan_and_user,how='outer',on='uid')
    train_feature_month_uid_all_replace_none = feature_month_is_loan_sum[:int(len(feature_month_is_loan_sum['uid'])*0.8)]

    return train_feature_month_uid_all_replace_none

def build_test_set():
    loan_and_user = pd.merge(file_loan_sum,file_user,how='outer',on='uid')
    loan_and_user = loan_and_user.sort_values(by='uid')
    loan_and_user['loan_sum'] = loan_and_user['loan_sum']/10
    del loan_and_user['age']
    del loan_and_user['sex']
    del loan_and_user['limit']
    del loan_and_user['active_date']
    # 取前80%作为训练集
    feature_month_is_loan_sum = pd.merge(feature_month_uid_all_replace_none , loan_and_user,how='outer',on='uid')
    test_feature_month_uid_all_replace_none = feature_month_is_loan_sum[int(len(feature_month_is_loan_sum['uid'])*0.8)+1:]
    del test_feature_month_uid_all_replace_none['loan_sum']
    # 替换所有nan为0
    build_test_set = test_feature_month_uid_all_replace_none.where(test_feature_month_uid_all_replace_none.notnull(), 0)
    return build_test_set

build_train_set = build_train_set()

build_test_set = build_test_set()

build_train_set.to_csv(generate_path + 'build_train_set.csv')

build_test_set.to_csv(generate_path + 'build_test_set.csv')

# dtrain = xgb.DMatrix(generate_path + 'build_train_set.libsvm')

# dtest = xgb.DMatrix(generate_path + 'build_test_set.libsvm')

In [11]:
feature_month_uid_is_loan_all

,uid,click_count_8,price_by_max_qty_8,qty_max_8,discount_by_max_qty_8,age_8,sex_8,limit_8,loan_count_8,loan_sum_8,...,qty_max_11,discount_by_max_qty_11,age_11,sex_11,limit_11,loan_count_11,loan_sum_11,loan_sum_mean_11,is_loan_11,diff_11
0,4,50.0,14.665733,4.0,11.159862,30.0,1.0,0.529215,2.0,1.044569,...,1.0,0.000000,30.0,1.0,0.529215,1.0,0.472302,0.472302,1.0,0.569137
1,20,29.0,18.710273,8.0,16.992488,35.0,1.0,0.629205,6.0,3.057334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,23,79.0,25.752178,9.0,15.675134,35.0,1.0,0.572277,5.0,1.931297,...,2.0,0.000000,35.0,1.0,0.572277,1.0,0.386259,0.386259,1.0,1.860174
3,28,33.0,9.793121,3.0,6.589657,25.0,1.0,0.615341,1.0,0.429265,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,39,93.0,32.888237,12.0,17.958687,35.0,1.0,0.629205,1.0,0.429265,...,1.0,0.000000,35.0,1.0,0.629205,7.0,2.930183,0.418598,1.0,2.106079
5,43,78.0,5.733840,2.0,0.000000,40.0,1.0,0.615341,1.0,0.572277,...,3.0,4.460755,40.0,1.0,0.615341,1.0,0.529215,0.529215,1.0,0.861260
6,53,7.0,0.000000,0.0,0.000000,25.0,1.0,0.529215,1.0,0.540542,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,64,86.0,2.795889,1.0,1.489896,25.0,1.0,0.529215,1.0,0.472302,...,5.0,0.000000,25.0,1.0,0.529215,1.0,0.386259,0.386259,1.0,1.429559
8,68,32.0,4.951210,1.0,0.000000,25.0,1.0,0.529215,7.0,2.982971,...,8.0,4.876574,25.0,1.0,0.529215,2.0,1.055898,0.527949,1.0,0.012664
9,69,14.0,33.032845,10.0,2.926196,40.0,1.0,0.615341,1.0,0.552725,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
label = np.random.randint(2, size=5)

In [8]:
dtrain = xgb.DMatrix(build_train_set, label = label)

In [ ]:
dtrain

In [35]:
from sklearn.cross_validation import train_test_split
train_xy,val = train_test_split(build_train_set, test_size = 0.3,random_state=1)
